In [6]:
import os, json
import markdown
from pathlib import Path
from langchain_text_splitters import MarkdownHeaderTextSplitter

OUTPUT_DIR = Path("./output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

document_mk_file = "../finish/JEGO.md"
car_model = Path(document_mk_file).stem.split("(")[0]

with open(document_mk_file, "r", encoding="utf-8") as f:
    markdown_document = f.read()

headers_to_split_on = [
    ("#", "#"),
    ("##", "##"),
    ("###", "###"),
    ("####", "####"),
    ("#####", "#####"),
]

In [7]:
# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

md_header_splits[:5]

[Document(page_content='# Gogoro Jego  \n發佈日期:2024.02.29 適用車款:\nJEGO\niQ System® 版本:6.8 以後 Gogoro® App 版本:3.3 以後 製造商:睿能創意股份有限公司 地址:桃園市龜山區頂湖路 33 號 電話:03-273-0900 **客服中心電話:**0800-365-996 Copyright© 2021 睿能創意股份有限公司 **著作權所有,並保留一切權利。本車主手冊中任何部分未經睿能創意股份有限公司**\n事前書面同意,不得以任何形式轉載、複製或拷貝。', metadata={'#': 'Gogoro Jego'}),
 Document(page_content='## 目錄\n###### 2\n| 目錄                                             |    |\n|--------------------------------------------------|----|\n| 1. 在您上路之前                                  | 3  |\n| 1.1 如何使用本手冊                               | 3  |\n| 1.2 安全提醒                                     | 4  |\n| 1.3 每次騎乘前應檢查項目                         | 5  |\n| 2. 認識您的 Smartscooter® 智慧電動機車           | 6  |\n| 2.1 JEGO 系列各部位名稱                          | 6  |\n| 2.2 左把手                                       | 7  |\n| 2.3 儀表板                                       | 8  |\n| 3. 準備上路                                      | 9  |\n| 3.1 開啟系統電源並解鎖龍頭                       | 9  |\n| 3.2 

In [14]:
import re
from langchain.schema import Document

def clean_context(text):
    text = re.sub(r'\s{2,}', '  ', text)  # Replace multiple spaces with two spaces
    return text

def extract_page_metadata(text):
    page_metadata = None
    cleaned_lines = []
    page_pattern = re.compile(r'^###### (\d+)')
    header_pattern = re.compile(r'^(#{1,5}) ')

    for line in text.split('\n'):
        match = page_pattern.match(line)
        if match:
            page_metadata = int(match.group(1))
        elif not header_pattern.match(line):
            cleaned_lines.append(line)

    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text, page_metadata

# Function to replace image paths in the content
def replace_image_paths(text):
    new_text = re.sub(r'!\[(.*?)\]\((.*?)\)', rf'![\1](images/{car_model}/\2)', text)
    return new_text

processed_splits = []
for doc in md_header_splits:
    content, page_metadata = extract_page_metadata(doc.page_content)

    content = replace_image_paths(content)

    cleaned_content = clean_context(content)

    new_metadata = doc.metadata.copy()
    # new_metadata['car_model'] = car_model
    if page_metadata:
        new_metadata['page'] = page_metadata

    processed_splits.append(Document(page_content=cleaned_content, metadata=new_metadata))
processed_splits[:5]

[Document(page_content='發佈日期:2024.02.29 適用車款:\nJEGO\niQ System® 版本:6.8 以後 Gogoro® App 版本:3.3 以後 製造商:睿能創意股份有限公司 地址:桃園市龜山區頂湖路 33 號 電話:03-273-0900 **客服中心電話:**0800-365-996 Copyright© 2021 睿能創意股份有限公司 **著作權所有,並保留一切權利。本車主手冊中任何部分未經睿能創意股份有限公司**\n事前書面同意,不得以任何形式轉載、複製或拷貝。', metadata={'#': 'Gogoro Jego'}),
 Document(page_content='| 目錄  |  |\n|--------------------------------------------------|----|\n| 1. 在您上路之前  | 3  |\n| 1.1 如何使用本手冊  | 3  |\n| 1.2 安全提醒  | 4  |\n| 1.3 每次騎乘前應檢查項目  | 5  |\n| 2. 認識您的 Smartscooter® 智慧電動機車  | 6  |\n| 2.1 JEGO 系列各部位名稱  | 6  |\n| 2.2 左把手  | 7  |\n| 2.3 儀表板  | 8  |\n| 3. 準備上路  | 9  |\n| 3.1 開啟系統電源並解鎖龍頭  | 9  |\n| 3.2 關閉系統電源並上鎖龍頭  | 9  |\n| 3.3 開啟座墊下置物箱  | 9  |\n| 4. 上路騎乘  | 9  |\n| 4.1 取車及架車  | 10 |\n| 4.2 啟動及關閉馬達  | 11 |\n| 4.2.1 啟動馬達  | 11 |\n| 4.2.2 關閉馬達  | 11 |\n| 4.3 前進及後退  | 12 |\n| 4.3.1 電子油門  | 12 |\n| 4.3.2 電子倒車油門  | 12 |\n| 4.4 減速及停止  | 13 |\n| 4.4.1 SBS 同步煞車系統  | 13 |\n| 4.5 低電量模式  | 13 |\n| 5. 補充電力  | 14 |\n| 5.1 在 GoStation® 電池交換站交換電池  | 14 |\n| 5.1.1 用 Gogoro

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 400
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(processed_splits)
splits[:5]

[Document(page_content='發佈日期:2024.02.29 適用車款:\nJEGO\niQ System® 版本:6.8 以後 Gogoro® App 版本:3.3 以後 製造商:睿能創意股份有限公司 地址:桃園市龜山區頂湖路 33 號 電話:03-273-0900 **客服中心電話:**0800-365-996 Copyright© 2021 睿能創意股份有限公司 **著作權所有,並保留一切權利。本車主手冊中任何部分未經睿能創意股份有限公司**\n事前書面同意,不得以任何形式轉載、複製或拷貝。', metadata={'#': 'Gogoro Jego'}),
 Document(page_content='| 目錄  |  |\n|--------------------------------------------------|----|\n| 1. 在您上路之前  | 3  |\n| 1.1 如何使用本手冊  | 3  |\n| 1.2 安全提醒  | 4  |\n| 1.3 每次騎乘前應檢查項目  | 5  |\n| 2. 認識您的 Smartscooter® 智慧電動機車  | 6  |\n| 2.1 JEGO 系列各部位名稱  | 6  |\n| 2.2 左把手  | 7  |\n| 2.3 儀表板  | 8  |\n| 3. 準備上路  | 9  |\n| 3.1 開啟系統電源並解鎖龍頭  | 9  |\n| 3.2 關閉系統電源並上鎖龍頭  | 9  |\n| 3.3 開啟座墊下置物箱  | 9  |\n| 4. 上路騎乘  | 9  |\n| 4.1 取車及架車  | 10 |', metadata={'#': 'Gogoro Jego', '##': '目錄', 'page': 2}),
 Document(page_content='| 4. 上路騎乘  | 9  |\n| 4.1 取車及架車  | 10 |\n| 4.2 啟動及關閉馬達  | 11 |\n| 4.2.1 啟動馬達  | 11 |\n| 4.2.2 關閉馬達  | 11 |\n| 4.3 前進及後退  | 12 |\n| 4.3.1 電子油門  | 12 |\n| 4.3.2 電子倒車油門  | 12 |\n| 4.4 減速及停止  | 13 |\n| 

In [16]:
output_dir = OUTPUT_DIR / car_model
os.makedirs(output_dir, exist_ok=True)

for i, doc in enumerate(splits):
    header_data = []
    for header, value in doc.metadata.items():
        # if header in ['#', '##', '###', '####', '#####']:
        if re.match(r'#+$', header):
            header_data.append(f"{header} {value}")
        else:
            header_data.append(f"<{header}>{value}</{header}>")
    combined_content = "\n".join(header_data)
    combined_content += "\n" + doc.page_content

    with open(os.path.join(output_dir, f"split_{i}.txt"), "w", encoding="utf-8") as f:
        f.write(combined_content)
    
    metadata = doc.metadata.copy()
    metadata["car_model"] = car_model.upper()
    metadata = {"metadataAttributes": metadata}
    
    with open(os.path.join(output_dir, f"split_{i}.txt.metadata.json"), "w", encoding="utf-8") as f:
        
        json.dump(metadata, f, indent=2, ensure_ascii=False)